In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
# import bert
# from bert import BertModelLayer
import functools

from tensorflow import keras
from tensorflow.keras import preprocessing
import tensorflow as tf
import numpy as np
import os

import tensorflow_text as tftext
import sentencepiece as spm
import sys

import json
import math
import os
import csv
import collections
import pandas as pd

from absl import app
from absl import flags
# from absl import logging
import logging
import tensorflow as tf
import tensorflow_hub as hub

sys.path.extend(["/space/users/jgeorge/git/tensorflow_experiments/tf2.3/models"])

# pylint: disable=g-import-not-at-top,redefined-outer-name,reimported
# from official.modeling import model_training_utils

from official.nlp.modeling.models import bert_classifier, bert_pretrainer
# from official.nlp.modeling.models.bert_classifier import BertClassifier
# from official.nlp.modeling.models.bert_pretrainer import BertPretrainer
# from official.nlp import bert_modeling as modeling
# from official.nlp import bert_models
from official.nlp import optimization
from official.nlp.bert import common_flags
from official.nlp.bert import input_pipeline
from official.nlp.bert import model_saving_utils
from official.utils.misc import distribution_utils
from official.utils.misc import keras_utils
from official.nlp.bert import tokenization

from official.nlp.albert import configs as albert_configs
from official.nlp.bert import run_classifier as run_classifier_bert

# from tensorflow.python.keras.engine import network

from tensorflow.python.keras.utils import tf_utils
from sklearn.model_selection import train_test_split

In [2]:
# https://stackoverflow.com/questions/13733552/logger-configuration-to-log-to-file-and-print-to-stdout
log_filepath = "/var/extra/users/jgeorge/tf2.0/input/dish/multi_logs.txt"
file_handler = logging.handlers.RotatingFileHandler(log_filepath, maxBytes=(1048576*5), backupCount=7)
# file_handler = logging.FileHandler()
formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
# logging.get_absl_handler().use_absl_log_file('absl_logging', os.path.join(main_data_folder,'Smithfield/AlwaysOn/logs'))
file_handler.setFormatter(formatter)

rootLogger = logging.getLogger()
# customLogger = logging.getLogger('TestControl')
rootLogger.setLevel(logging.INFO)
# removing existing loggers
while(len(rootLogger.handlers)>0):
    rootLogger.removeHandler(rootLogger.handlers[0])

# uncomment to add logs to a file    
# rootLogger.addHandler(file_handler)

stream_handler = logging.StreamHandler(sys.stdout)
rootLogger.addHandler(stream_handler)
print('logging info',rootLogger.handlers )


logging info [<StreamHandler stdout (NOTSET)>]


In [3]:
FLAGS = flags.FLAGS    

In [4]:
print(tf.__version__)

2.3.0


In [5]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:4', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:5', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:6', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:7', device_type='GPU')]

In [6]:
##restricting no of gpus
gpus = tf.config.list_physical_devices('GPU')
device_to_use = gpus[3]
tf.config.experimental.set_memory_growth(device_to_use,True)
tf.config.experimental.set_visible_devices(device_to_use, 'GPU')
print(tf.config.get_visible_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')]


In [7]:
##setting direcory for downloading tfhub modules
os.environ["TFHUB_CACHE_DIR"] = '/space/engineering/tfhub_modules'

In [8]:
dish_data_path='/var/extra/users/jgeorge/tf2.0/input/dish/data/jan17_2020/dishDataJan17.txt'
df = pd.read_csv(dish_data_path,sep='\t',header=None,names=['filename','text','granular_intent','ru_intent'])

In [9]:
df.head()

,filename,text,granular_intent,ru_intent
0,INT-sv1appis14-1504137880316-305603_4567,can you send my bill to my mail?,billing-preferences,billing-preferences
1,INT-va1appis15-1504373018548-41332,My Wally receiver has lost Satellite signal in...,comp_part_signal_loss-issue,comp_part_signal_loss-issue
2,INT-sv1appis12-1503954587819-263368,I need a payment extension so i don't get my s...,payment_extension-request,payment_extension-request
3,8ed8e9b1-10f5-438d-c409-e616c3ff9ede,how can i find my local channels. it seems i d...,channel_package-issue,channel_package-issue
4,INT-sv1appis13-1504099917638-293735,Wanted to speak with someone about my bill,representative-request,representative-request


In [10]:
df_train,df_val,y_train,y_val = train_test_split(df,df['granular_intent'],train_size=0.9,random_state=42,stratify=df['granular_intent'])

In [11]:
# main_exp_folder = '/space/users/jgeorge/projects/k/tensorflow2-question-answering/input/dish/data/jan17_2020/'
main_exp_folder = '/var/extra/users/jgeorge/tf2.0/input/dish/data/jan17_2020/'

In [12]:
output_folder = '/var/extra/users/jgeorge/tf2.0/output/dish/models/bert_multilingual'

In [13]:
train_csv_file='/var/extra/users/jgeorge/tf2.0/input/dish/data/jan17_2020/train.txt'
eval_csv_file='/var/extra/users/jgeorge/tf2.0/input/dish/data/jan17_2020/val.txt'

df_train.to_csv(train_csv_file,sep='\t',header=False,index=False)
df_val.to_csv(eval_csv_file,sep='\t',header=False,index=False)

### Writing a file with list of intents

In [14]:
intent_file = '/var/extra/users/jgeorge/tf2.0/input/dish/data/jan17_2020/intentlist.txt'

In [15]:
with open(intent_file,'w',encoding='utf-8') as out_f:
    for intent in sorted(df_train['granular_intent'].unique()):
        out_f.write(intent+'\n')


In [16]:
intent_list = []
with open(intent_file,'r') as inp_f:
    for intent in inp_f:
        intent_list.append(intent.strip())

In [17]:
len(df_train)

23146

In [18]:
len(df_val)

2572

In [19]:
input_meta_data = {
    'max_seq_length':128,
    'num_labels':len(intent_list),
    'train_data_size':len(df_train),
    'eval_data_size':len(df_val),
    'default_label':'other-other'
}

In [20]:
input_meta_data

{'max_seq_length': 128,
 'num_labels': 142,
 'train_data_size': 23146,
 'eval_data_size': 2572,
 'default_label': 'other-other'}

In [21]:
input_meta_path = os.path.join(main_exp_folder,'input_metadata.json')

In [22]:
#writing metadata file
with open(input_meta_path,'w',encoding='utf-8') as jf:
    jf.write(json.dumps(input_meta_data))

In [23]:
#reading metadata file
with open(input_meta_path,'r',encoding='utf-8') as jf:
#     input_meta_data = json.loads(jf.read())
    input_meta_data = json.load(jf)

## Data preprocessing & feeding into graph
We will be reading the regular text file as input.   
If we want to save time during training, we could tokenize the data upfront & store it as tfrecords    

Writing as tfrecord is not necessary, but it is a more optimized file format for reading into 
tf.data api (you can even base your tf dataset on a textfile or even a python iterator)
Also it's not necessary to use tf.data apis but this is much more optimized, like preloading data into gpu memory & optimization required if you are running across systems & all

In [24]:
max_seq_length=128
bert_config = {}
bert_config["initializer_range"] = 0.02
# we will use this for the Dropout probability between bert layer & final Dense layer
bert_config["hidden_dropout_prob"]=0.2
num_labels = input_meta_data['num_labels']
default_label = input_meta_data['default_label']
delimiter = '\t'


In [25]:
output_folder = '/var/extra/users/jgeorge/tf2.0/output/dish/models/bert_multilingual/'

### Input Dataset creation 

#### Layer to convert label to id

In [28]:
class LabelToId(tf.keras.layers.Layer):
#     def __init__(self,model_proto):
    def __init__(self,filepath,default_index,*args,**kwargs):
        """ 
        filepath - path to the text file with 1 label per line
        """
        super(LabelToId, self).__init__(trainable=False,dtype=tf.string,*args, **kwargs)
        self.filepath = filepath
        self.default_index=default_index
        
    def build(self,input_shape):
        # need to convert label name to label index. So keeping line no as value & whole text line as key
        # issue with using table lookup (both file based & key value tensorbased) in tf2.2 (tf2.x) - https://github.com/tensorflow/tensorflow/issues/38305
        self.table = tf.lookup.StaticHashTable(
            tf.lookup.TextFileInitializer(self.filepath,
                                          key_dtype=tf.string,key_index=tf.lookup.TextFileIndex.WHOLE_LINE,
                                     value_dtype=tf.int64,value_index=tf.lookup.TextFileIndex.LINE_NUMBER),self.default_index)
        self.built=True
        
    def call(self, input_index):
        word_ids = self.table.lookup(input_index)
        return word_ids
    
    def get_config(self):
        config = super(LabelToId, self).get_config()
#         config.update({'filepath': self.filepath})
        return config


In [29]:

def get_input_dataset(file_path, default_label, label_path, batch_size=16, delimiter = '\t', is_training=True):
    dataset = tf.data.experimental.CsvDataset(file_path, record_defaults=[tf.constant(''),tf.constant('MISSING')],
                                                   header=False,field_delim=delimiter,use_quote_delim=True,
                                                   select_cols=[1,2])
    label_dict = {}
    with open(label_path) as f:
        for i,line in enumerate(f):
            label_dict[line.strip()] = i
        
    default_index = label_dict.get(default_label)
    label_to_id = LabelToId(intent_file,default_index)
    if is_training:
        dataset = dataset.shuffle(100)
        dataset = dataset.repeat()
    # batching data
    dataset = dataset.batch(batch_size)
    # updating only label, text will be processed as part of training graph
    dataset = dataset.map(lambda text, label: (text, label_to_id(label)),num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return dataset

### creating a train dataset just to try out

In [31]:
batch_size = 16
training_dataset = get_input_dataset(train_csv_file, default_label, intent_file, batch_size, delimiter,True)
# evaluation_dataset = get_input_dataset(eval_csv_file, default_label, intent_file, batch_size, delimiter,False)
for line in training_dataset.take(1):
    print(line)

(<tf.Tensor: shape=(16,), dtype=string, numpy=
array([b'I would like to know if there any discounts available',
       b'where is the channel line up', b'I would like to update my mail',
       b'trying to change autopay', b'Removing a ppv',
       b'I need to cancel my appointment... For today',
       b'Smart package with heartland and my $3 Starz',
       b'can I get my lineup of channels',
       b'Get an extenstion and take off add ons',
       b'I want my cable to be taken off for good',
       b'got my new remote from dish yesterday. have not had any luck getting it to sync with my receiver. i have went thru both versions of trying to get the remote to sync and have had no luck.',
       b'We have an appointment for tomorrow and want to know if we can add another room for no additional charge. Direct has an offer for 4 rooms included in their package at $50.00 per month 150 channels',
       b'I need to buy a remote', b'need help setting my remote with tv ',
       b'DO YOU OFFE

<ipython-input-29-1cb11528cf86>:18: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  dataset = dataset.map(lambda text, label: (text, label_to_id(label)),num_parallel_calls=tf.data.experimental.AUTOTUNE)


## Model code, graph creation
This section is the core logic of the model, here we are using tensorflow hub url for albert model. Using hub simplifies the code a lot      
It's a slightly simplied version of official bert code, that code have functionality to load a non hub model. 
#### refer to models/official/nlp/bert/run_classifier.py
Since in this notebook we are only using one gpu, we can even ignore the strategy part

In [50]:
# strategy = distribution_utils.get_distribution_strategy('mirrored',num_gpus=1)

# strategy = tf.distribute.OneDeviceStrategy("device:GPU:2")
#since devices to use is set to 2 already, only 1 device is visible which is 0
strategy = tf.distribute.OneDeviceStrategy("device:GPU:0")

### Temporary tryouts

### Sentence piece encoding & other preprocessing for text

In [32]:
bert_preprocessor_url = "https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/1"
bert_preprocessor_loaded = hub.load(bert_preprocessor_url)

Using /space/engineering/tfhub_modules to cache modules.


### bert model laoding

In [33]:
# multilingual bert hub url
bert_hub_url = "https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3"

We need to load the used sentence piece model from the hub module, 
it will be present in the assests folder of the downloaded albert model
Since we are using hub here & not directly downloading it, the path can be fetched from hub layer

In [34]:
bert_hub_loaded = hub.load(bert_hub_url)
vocab_path = bert_hub_loaded.vocab_file.asset_path.numpy().decode('utf-8')
print(f'vocab_path {vocab_path}')
# or if you are using hub.KeraLayer
# bert_model = hub.KerasLayer(bert_hub_url,trainable=True)
# sentencepiece_path = bert_model.resolved_object.sp_model_file.asset_path.numpy()

##if you have directly downloaded the model you could do this
# sentencepiece_path = os.path.join(model_dir, "assets", "30k-clean.model")


vocab_path /space/engineering/tfhub_modules/67c5e34716a3dd645ca3507fac096a07b2c8607d/assets/vocab.txt


### Defining the graph
#### trying out the multilingual bert model

In [35]:
input_text = tf.keras.layers.Input(shape=(), dtype=tf.string,name='input_text')
tokenize = hub.KerasLayer(bert_preprocessor_loaded.tokenize)
tokenized_inputs = [tokenize(input_text)] # bert can handle 2 input_texts (segments)
 
bert_pack_inputs = hub.KerasLayer(bert_preprocessor_loaded.bert_pack_inputs,arguments=dict(seq_length=max_seq_length))
encoder_inputs = bert_pack_inputs(tokenized_inputs)
encoder = hub.KerasLayer(bert_hub_url,trainable=True)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]      # [batch_size, 768].
sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 768].
bert_output = tf.keras.layers.Dropout(rate=bert_config["hidden_dropout_prob"])(pooled_output) 
initializer = tf.keras.initializers.TruncatedNormal(stddev=bert_config["initializer_range"])
output = tf.keras.layers.Dense(num_labels,kernel_initializer=initializer,name='output',dtype=tf.float32)(bert_output)
    
# model = tf.keras.Model(inputs={'input_ids':input_ids,
#                                  'input_mask':input_mask,
#                                  'segment_ids':segment_ids},
#                          outputs=output),bert_model

model = tf.keras.Model(inputs=input_text,
                         outputs=output)

In [36]:
encoder_inputs

{'input_word_ids': <tf.Tensor 'keras_layer_1/StatefulPartitionedCall:2' shape=(None, None) dtype=int32>,
 'input_mask': <tf.Tensor 'keras_layer_1/StatefulPartitionedCall:0' shape=(None, None) dtype=int32>,
 'input_type_ids': <tf.Tensor 'keras_layer_1/StatefulPartitionedCall:1' shape=(None, None) dtype=int32>}

### writing a function to get training model graph

In [37]:
def get_multilingual_bert(bert_config,num_labels,max_seq_length,model_hub_url,preprocessor_hub_url):
    bert_preprocessor_loaded = hub.load(preprocessor_hub_url)
    input_text = tf.keras.layers.Input(shape=(), dtype=tf.string,name='input_text')
    tokenize = hub.KerasLayer(bert_preprocessor_loaded.tokenize)
    tokenized_inputs = [tokenize(input_text)] # bert can handle 2 input_texts (segments)
    
    bert_pack_inputs = hub.KerasLayer(bert_preprocessor_loaded.bert_pack_inputs,arguments=dict(seq_length=max_seq_length))
    encoder_inputs = bert_pack_inputs(tokenized_inputs)
    encoder = hub.KerasLayer(model_hub_url,trainable=True)
    outputs = encoder(encoder_inputs)
    pooled_output = outputs["pooled_output"]      # [batch_size, 768].
    sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 768].
    bert_output = tf.keras.layers.Dropout(rate=bert_config["hidden_dropout_prob"])(pooled_output) 
    initializer = tf.keras.initializers.TruncatedNormal(stddev=bert_config["initializer_range"])
    output_dense = tf.keras.layers.Dense(num_labels,kernel_initializer=initializer,name='output',dtype=tf.float32)(bert_output)

    return tf.keras.Model(inputs=input_text,outputs=output_dense),encoder

In [60]:
## clearing the existing graphs
tf.keras.backend.clear_session()

### Let's look at the classification model description/summary
We are calling get_albert_model here just to print out model summary, otherwise we call it internally
in another function

In [38]:
classification_model,core_bert_model = get_multilingual_bert(bert_config=bert_config,
                                                              num_labels=input_meta_data['num_labels'],
                                                              max_seq_length=max_seq_length,
                                                               model_hub_url=bert_hub_url,
                                                                preprocessor_hub_url=bert_preprocessor_url)

In [39]:
classification_model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_text (InputLayer)         [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer_3 (KerasLayer)      (None, None, None)   0           input_text[0][0]                 
__________________________________________________________________________________________________
keras_layer_4 (KerasLayer)      {'input_word_ids': ( 0           keras_layer_3[0][0]              
__________________________________________________________________________________________________
keras_layer_5 (KerasLayer)      {'default': (None, 7 177853441   keras_layer_4[0][0]              
                                                                 keras_layer_4[0][1]   

In [41]:
core_bert_model.get_config()

{'name': 'keras_layer_5',
 'trainable': True,
 'dtype': 'float32',
 'handle': 'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3'}

#### Define Loss function,  metric

In [42]:
def get_loss_fn(num_classes, loss_factor=1.0):
    """Gets the classification loss function."""

    def classification_loss_fn(labels, logits):
        """Classification loss."""
        labels = tf.squeeze(labels)
#         log_probs = tf.nn.log_softmax(logits, axis=-1)
#         one_hot_labels = tf.one_hot(
#             tf.cast(labels, dtype=tf.int32), depth=num_classes, dtype=tf.float32)
#         per_example_loss = -tf.reduce_sum(
#             tf.cast(one_hot_labels, dtype=tf.float32) * probs, axis=-1)
        per_example_loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = labels, logits=logits)
        #batch loss
        loss = tf.reduce_mean(per_example_loss)
        loss *= loss_factor
        return loss
    return classification_loss_fn  


def metric_fn():
    return tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy',dtype=tf.float32)

#### Function for running training

In [43]:
def run_keras_compile_fit(model_dir,
                          strategy,
                          model_fn,
                          training_dataset,
                          evaluation_dataset,
                          loss_fn,
                          metric_fn,
                          init_checkpoint,
                          epochs,
                          steps_per_epoch,
                          steps_per_loop,
                          eval_steps,
                          custom_callbacks=None):
    """Runs BERT classifier model using Keras compile/fit API."""
    ###slightly simplied version of official bert code 
    # refer to models/official/nlp/bert/run_classifier.py -   function run_keras_compile_fit
    # if you running on a single gpu or on just cpu this strategy is not necessary
    with strategy.scope():
        #sub_model is the original bert
        classification_model, sub_model = model_fn()
        optimizer = classification_model.optimizer
        
        # this is not required for the hub version of the model, this restore method is trying to restor values,
        # from a saved bert model (since we have only provided the sub_model to checkpoint)
        # Let's saying we stopped training at some point & want to continue from that point next time,
        # here we can instead load our classification_model & init_checkpoint can be our previous saved checkpoint file
        if init_checkpoint:
            checkpoint = tf.train.Checkpoint(model=sub_model)
#             checkpoint = tf.train.Checkpoint(model=classification_model)
            checkpoint.restore(init_checkpoint).assert_existing_objects_matched()
#             checkpoint.restore(init_checkpoint).expect_partial()

        classification_model.compile(
            optimizer=optimizer,
            loss=loss_fn,
            metrics=[metric_fn()])
    #     ,experimental_steps_per_execution=steps_per_loop)

        summary_dir = os.path.join(model_dir, 'summaries')
        summary_callback = tf.keras.callbacks.TensorBoard(summary_dir, profile_batch='10,20')
        checkpoint_path = os.path.join(model_dir, 'checkpoint')
        checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
            checkpoint_path, save_best_only=True, save_weights_only=True)
        early_stopping = tf.keras.callbacks.EarlyStopping()

        if custom_callbacks is not None:
            custom_callbacks += [summary_callback, checkpoint_callback, early_stopping]
        else:
            custom_callbacks = [summary_callback, checkpoint_callback, early_stopping]
#       Note that we are only passing x & not y in fit function
#       Refer to keras model.fit documentation for further details

#       If `x` is a dataset, generator,
#       or `keras.utils.Sequence` instance, `y` should
#       not be specified (since targets will be obtained from `x`).
        model_history = classification_model.fit(
            x=training_dataset,
            validation_data=evaluation_dataset,
            steps_per_epoch=steps_per_epoch,
            epochs=epochs,
            validation_steps=eval_steps,
            callbacks=custom_callbacks)

        return classification_model

## Training

In [44]:
## clearing the existing graphs
tf.keras.backend.clear_session()

### defining various parameters for the experiment

In [45]:
steps_per_loop = 1
learning_rate=1e-5
epochs=20

train_batch_size=16
eval_batch_size=16

train_data_size = input_meta_data['train_data_size']
steps_per_epoch = int(train_data_size / train_batch_size)
print(steps_per_epoch)

warmup_steps = int(epochs * train_data_size * 0.1 / train_batch_size)
eval_steps = int(math.ceil(input_meta_data['eval_data_size'] / eval_batch_size))

1446


### training & validation dataset object creation

In [46]:

training_dataset = get_input_dataset(train_csv_file, default_label, intent_file, train_batch_size, delimiter, is_training=True)
evaluation_dataset = get_input_dataset(eval_csv_file, default_label, intent_file, train_batch_size, delimiter, is_training=False)
## taking only 1 row & iterating & viewing output
for line in training_dataset.take(1):
    print(line)

(<tf.Tensor: shape=(16,), dtype=string, numpy=
array([b'my hopper is not working',
       b'how can I cancel the final four games',
       b'I don\xe2\x80\x99t want to add hbo',
       b'where is the channel line up',
       b'I am wondering when I will be reimbursed for my DISH Equipment? I was charged for not returning the equipment within 30 days but have sense sent it out about 2 weeks ago...',
       b'need help setting my remote with tv ',
       b'How do I cancel HBO| Showtime and whatever else is about to become an extra expense?',
       b"What's the difference between smart pack and flex pack? Can I just get local channels ,HBO Starz and Showtime?",
       b'i need to have my service added to my garage is this possible i have 3 joeys now',
       b'Is there any possible way my appointment could be around 5pm',
       b'Hi I think Im shutting down may account . My husband is not working and Its difficult to be paying $85. How can I be helped?',
       b'i don want march madnes

<ipython-input-29-1cb11528cf86>:18: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  dataset = dataset.map(lambda text, label: (text, label_to_id(label)),num_parallel_calls=tf.data.experimental.AUTOTUNE)


In [47]:
loss_multiplier = 1
max_seq_length = input_meta_data['max_seq_length']
num_labels = input_meta_data['num_labels']
loss_fn = get_loss_fn(num_labels,loss_multiplier)

def _get_classifier_model():
    classifier_model,core_model = get_multilingual_bert(bert_config=bert_config,
                                                              num_labels=num_labels,
                                                              max_seq_length=max_seq_length,
                                                               model_hub_url=bert_hub_url,
                                                                preprocessor_hub_url=bert_preprocessor_url)
    ##This is basically Adam optimizer with weight decay after set no of warm up steps & 
    # before that an increasing learning rate from 0 to initial_lr
    # refer to models/official/nlp/optimization.py for more details
    classifier_model.optimizer = optimization.create_optimizer(init_lr=learning_rate,
                                                               num_train_steps=steps_per_epoch*epochs,
                                                               num_warmup_steps=warmup_steps)
    return classifier_model,core_model


In [51]:
## clearing the existing graphs
tf.keras.backend.clear_session()

In [52]:

trained_model = run_keras_compile_fit(model_dir=output_folder,strategy=strategy,model_fn=_get_classifier_model,
                                     training_dataset=training_dataset,evaluation_dataset=evaluation_dataset,
                                      loss_fn=loss_fn,metric_fn=metric_fn,
                                     init_checkpoint=None,
                                      epochs=epochs,
                                      steps_per_epoch=steps_per_epoch,
                                      steps_per_loop=steps_per_loop,
                                      eval_steps=eval_steps
                                     )

5 out of the last 5 calls to <function recreate_function.<locals>.restored_function_body at 0x7f04fd5cfa60> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/tutorials/customization/performance#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
6 out of the last 6 calls to <function recreate_function.<locals>.restored_function_body at 0x7f04fd5ccaf0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2)

### trying training for a few more epochs

## Saving the model
We don't want to have dropout in the network.
For the output have a softmax & a table lookup to on labels to return the label name 

In [316]:
class LabelLookup(tf.keras.layers.Layer):
#     def __init__(self,model_proto):
    def __init__(self,filepath,default,*args,**kwargs):
        """ 
        filepath - path to the text file with 1 label per line
        """
        super(LabelLookup, self).__init__(trainable=False,dtype=tf.int64,*args, **kwargs)
        self.filepath = filepath
        self.default=default
        
    def build(self,input_shape):
        # need to convert label index to label name. So keeping line no as key & whole text line as label
        # issue with using table lookup (both file based & key value tensorbased) in tf2.2 (tf2.x) - https://github.com/tensorflow/tensorflow/issues/38305
        self.table = tf.lookup.StaticHashTable(
            tf.lookup.TextFileInitializer(self.filepath,
                                          key_dtype=tf.int64,key_index=tf.lookup.TextFileIndex.LINE_NUMBER,
                                     value_dtype=tf.string,value_index=tf.lookup.TextFileIndex.WHOLE_LINE),self.default)
        self.built=True
        
    def call(self, input_index):
        word_ids = self.table.lookup(input_index)
        return word_ids
    
    def get_config(self):
        config = super(LabelLookup, self).get_config()
#         config.update({'filepath': self.filepath})
        return config


In [ ]:
tf.keras.layers.Layer()

In [443]:
class BertTokenizerLayer(tf.keras.layers.Layer):
    def __init__(self,filepath,default,*args,**kwargs):
        """ 
        filepath - path to the text file with 1 label per line
        """
        super(BertTokenizerLayer, self).__init__(trainable=False,dtype=tf.int32,*args, **kwargs)
        self.filepath = filepath
        self.default=default
        
    def build(self,input_shape):
        # need to convert label index to label name. So keeping line no as key & whole text line as label
        # issue with using table lookup (both file based & key value tensorbased) in tf2.2 (tf2.x) - https://github.com/tensorflow/tensorflow/issues/38305
        self.table = tf.lookup.StaticHashTable(
            tf.lookup.TextFileInitializer(self.filepath,
                                          key_dtype=tf.string,key_index=tf.lookup.TextFileIndex.WHOLE_LINE,
                                     value_dtype=tf.int64,value_index=tf.lookup.TextFileIndex.LINE_NUMBER),self.default)
        self.tokenizer = tftext.BertTokenizer(self.table,split_unknown_characters=False)
        self.built=True
        
    def call(self, word):
        word_ids = tf.cast(self.tokenizer.tokenize(word),dtype=self.dtype)
        return word_ids
    
    def get_config(self):
        config = super(BertTokenizerLayer, self).get_config()
#         config.update({'filepath': self.filepath})
        return config


In [438]:
vocab_file = '/space/engineering/tf_serve/models/dish_multilingual_bert/bert_multilingual_preprocessor/vocab.txt'
unk_word_index = 100
table = tf.lookup.StaticHashTable(
            tf.lookup.TextFileInitializer(vocab_file,
                                          key_dtype=tf.string,key_index=tf.lookup.TextFileIndex.WHOLE_LINE,
                                     value_dtype=tf.int64,value_index=tf.lookup.TextFileIndex.LINE_NUMBER),unk_word_index)
tokenizer = tftext.BertTokenizer(table,split_unknown_characters=False)
tokenizer

In [415]:
tokenizer.tokenize(tf.constant(["talk to agent", "representative"]))

<tf.RaggedTensor [[[31311], [10114], [18980]], [[33786]]]>

In [439]:
tokenize_layer = BertTokenizerLayer(vocab_file, unk_word_index)
temp = [tokenize_layer(input_text)]
temp

data type int32


[tf.RaggedTensor(values=tf.RaggedTensor(values=Tensor("bert_tokenizer_layer_3/Cast_1:0", shape=(None,), dtype=int32), row_splits=Tensor("bert_tokenizer_layer_3/WordpieceTokenizeWithOffsets/WordpieceTokenizeWithOffsets/WordpieceTokenizeWithOffsets:1", shape=(None,), dtype=int64)), row_splits=Tensor("bert_tokenizer_layer_3/RaggedFromRowSplits/control_dependency:0", shape=(None,), dtype=int64))]

In [440]:
temp[0].dtype

tf.int32

In [421]:
def get_multilingual_bert_for_saving(bert_config,num_labels,max_seq_length,
                                     model_hub_url,preprocessor_path,
                                     checkpoint_path,
                                     default_label,
                                    max_top_intents=2, unk_word_index=100):
    bert_preprocessor_loaded = hub.load(preprocessor_path)
    vocab_file = os.path.join(preprocessor_path,"assets","vocab.txt")
    
    input_text = tf.keras.layers.Input(shape=(), dtype=tf.string,name='input_text')
#     tokenize = hub.KerasLayer(bert_preprocessor_loaded.tokenize)
    tokenize = BertTokenizerLayer(vocab_file, unk_word_index)
    
    tokenized_inputs = [tokenize(input_text)] # bert can handle 2 input_texts (segments)
    
    bert_pack_inputs = hub.KerasLayer(bert_preprocessor_loaded.bert_pack_inputs,arguments=dict(seq_length=max_seq_length))
    encoder_inputs = bert_pack_inputs(tokenized_inputs)
    encoder = hub.KerasLayer(model_hub_url,trainable=False)
    outputs = encoder(encoder_inputs)
    pooled_output = outputs["pooled_output"]      # [batch_size, 768].
    sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 768].
    initializer = tf.keras.initializers.TruncatedNormal(stddev=bert_config["initializer_range"])
    dense_output = tf.keras.layers.Dense(num_labels,kernel_initializer=initializer,name='output',dtype=tf.float32)(pooled_output)
    
    # calculating softmax over the output layer
    probs = tf.nn.softmax(dense_output,axis=-1,name='scores')
    # sorting & taking only max_top_intents
    sorted_indexes = tf.cast(tf.argsort(probs,axis=-1,direction='DESCENDING'),tf.int64)[:,:max_top_intents]
    sorted_scores1 = tf.sort(probs,axis=-1,direction='DESCENDING',name="scores1")[:,:max_top_intents]
    # this identity operation was only required to get the name correctly as in tf2.3 the sort object's name was coming differently
    # https://github.com/tensorflow/tensorflow/issues/39398
    # https://stackoverflow.com/questions/38626424/how-to-assign-new-name-or-rename-an-existing-tensor-in-tensorflow
    sorted_scores = tf.identity(sorted_scores1,name='scores_sorted')

    labels = LabelLookup(filepath=intent_file,default=default_label,name="classes")(sorted_indexes)

    # loaded_model.compile()
    #     checkpoint = tf.train.Checkpoint(model=model)
    #     checkpoint.restore(checkpoint_path).assert_existing_objects_matched()
    
    model  = tf.keras.Model(inputs={'input_text':input_text},
                         outputs={"classes":labels,"scores":sorted_scores})
    # FIXME the following is a hack (not sure if it will create other issues)
    model.output_names = ['classes', 'scores']

#     checkpoint = tf.train.Checkpoint(model=model)
#     checkpoint.restore(checkpoint_path).assert_existing_objects_matched()
    model.load_weights(checkpoint_path).assert_existing_objects_matched()

    return model

In [449]:
preprocessor_path = '/space/engineering/tfhub_modules/7d11d5f8a107b5acd196d06446e8a878d630fe38'

In [365]:
bert_preprocessor_url

'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/1'

In [451]:
tf.keras.backend.clear_session()

In [444]:
classification_model_to_save = get_multilingual_bert_for_saving(bert_config=bert_config,
                                                              num_labels=input_meta_data['num_labels'],
                                                              max_seq_length=max_seq_length,
                                                               model_hub_url=bert_hub_url,
                                                                preprocessor_path=preprocessor_path,
                                                                checkpoint_path=tf.train.latest_checkpoint(output_folder),
                                                                default_label=default_label, max_top_intents=2)

11 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7f4bce00cd30> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/tutorials/customization/performance#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
11 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7f4bcdf90310> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop,

In [450]:
output = classification_model_to_save.predict(['','hello','talk to agent', 'remote not working'])
output

{'classes': array([[b'other-other', b'representative-request'],
        [b'greet-hello', b'autopay-cancel'],
        [b'representative-request', b'other-other'],
        [b'remote-setup', b'remote-place_order']], dtype=object),
 'scores': array([[0.10534582, 0.05661591],
        [0.07370817, 0.02759776],
        [0.33845136, 0.0372153 ],
        [0.8056617 , 0.02082952]], dtype=float32)}

In [329]:
classification_model_to_save.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_text (InputLayer)         [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        (None, None, None)   0           input_text[0][0]                 
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'input_type_ids': ( 0           keras_layer[0][0]                
__________________________________________________________________________________________________
keras_layer_2 (KerasLayer)      {'default': (None, 7 177853441   keras_layer_1[0][0]              
                                                                 keras_layer_1[0][1]   

In [445]:
# final_model_path=os.path.join(output_folder,'final_model')
final_model_path='/space/engineering/tf_serve/models/dish_multilingual_bert'
final_versioned_model_path = os.path.join(final_model_path,'2')
classification_model_to_save.save(final_versioned_model_path)

INFO:tensorflow:Assets written to: /space/engineering/tf_serve/models/dish_multilingual_bert/2/assets
Assets written to: /space/engineering/tf_serve/models/dish_multilingual_bert/2/assets


### loading the saved model with preprocessing & trying it out

In [393]:
tf.keras.backend.clear_session()

In [446]:
loaded_final_model = tf.saved_model.load(final_versioned_model_path)

11 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7f4bbacfe940> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/tutorials/customization/performance#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
11 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7f4bbad61280> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop,

In [447]:
serving_default = loaded_final_model.signatures['serving_default']
serving_default

<ConcreteFunction signature_wrapper(input_text) at 0x7F4BBB34D700>

In [448]:
serving_default(tf.constant(['','hello','talk to agent', 'remote not working']))

{'scores': <tf.Tensor: shape=(4, 2), dtype=float32, numpy=
 array([[0.10534582, 0.05661591],
        [0.07370817, 0.02759776],
        [0.33845136, 0.0372153 ],
        [0.8056617 , 0.02082952]], dtype=float32)>,
 'classes': <tf.Tensor: shape=(4, 2), dtype=string, numpy=
 array([[b'other-other', b'representative-request'],
        [b'greet-hello', b'autopay-cancel'],
        [b'representative-request', b'other-other'],
        [b'remote-setup', b'remote-place_order']], dtype=object)>}

### running against a held out test set
ideally we need a held out test set, but using the validation set again just to have the testing code here

In [535]:
df_test = df_val.copy()

In [140]:
df_val.head()

,filename,text,granular_intent,ru_intent
20421,a9cf6bee-3971-4516-459f-2243d9a606fd,I want to inform you that I don't have electri...,billing-issue,billing-issue
22946,0f050cf9-88d7-44bd-23c8-5f26674f5b6a,What happen to my bill,billing-query,billing-query
8207,INT-sv1appis11-1503849377195-243917,I need to replace my remote control.,remote-place_order,remote-place_order
14144,INT-sv1appis14-1504137880316-305603_1158,Reciever is saying not authorized on every cha...,receiver_issue-get_help,receiver_issue-get_help
8164,6e1842f4-63a1-4259-3999-6928033ce7eb,i need to update my credit card info,payment-get_help,payment-get_help


In [206]:
output = classification_model_to_save.predict(df_val['text'].array[:20].to_numpy())
output

{'classes': array([[b'billing-issue', b'price-increase', b'billing-vague', ...,
         b'app-query', b'dvr-query', b'receiver_upgrade-query'],
        [b'billing-issue', b'billing-vague', b'billing-query', ...,
         b'ppv-cancel', b'order-query', b'receiver_upgrade-query'],
        [b'remote-place_order', b'remote-order_status',
         b'remote-not_working', ..., b'get-help', b'billing-vague',
         b'other-other'],
        ...,
        [b'channel_package-issue', b'streaming-issues',
         b'comp_part_signal_loss-issue', ..., b'order-cancel',
         b'internet-price_query', b'appointment-cancel'],
        [b'price-increase', b'billing-issue',
         b'account_cancel-promotion_seek', ..., b'online_account-setup',
         b'dish_pause-cancel', b'billing-preferences'],
        [b'greet-hello', b'yes-give', b'no-give', ..., b'promotion-query',
         b'billing-preferences', b'account-address_change']], dtype=object),
 'scores': array([[9.90572035e-01, 1.36903033e-03, 7

In [536]:
def get_predicted_intent_per_row(row, classification_model, text_column='text',out_of_domain_intent='none_none'):
    text = row['text']
    try:
        output = classification_model.predict([text])
        # 1st index is for batch & batch size is 1, 2nd is the list of intents, taking the top intent
        row['predicted_intent'] = output['classes'][0][0]
        row['score'] = output['scores'][0][0]
    except Exception as e:
        print(f'exception occured, text {text} classifying as out of domain intent error {e}')
        row['predicted_intent']  = out_of_domain_intent
        row['score'] = 1
    return row

In [537]:
print(f'size of test data {len(df_test)}')
df_test = df_test.apply(lambda row:get_predicted_intent_per_row(row,classification_model_to_save,text_column='text'),axis=1)
df_test.head()

size of test data 2572


,filename,text,granular_intent,ru_intent,predicted_intent,score
20421,a9cf6bee-3971-4516-459f-2243d9a606fd,I want to inform you that I don't have electri...,billing-issue,billing-issue,b'billing-issue',0.990572
22946,0f050cf9-88d7-44bd-23c8-5f26674f5b6a,What happen to my bill,billing-query,billing-query,b'billing-issue',0.983889
8207,INT-sv1appis11-1503849377195-243917,I need to replace my remote control.,remote-place_order,remote-place_order,b'remote-place_order',0.939638
14144,INT-sv1appis14-1504137880316-305603_1158,Reciever is saying not authorized on every cha...,receiver_issue-get_help,receiver_issue-get_help,b'receiver_issue-get_help',0.598178
8164,6e1842f4-63a1-4259-3999-6928033ce7eb,i need to update my credit card info,payment-get_help,payment-get_help,b'payment-get_help',0.947868


In [94]:
serving_default.structured_input_signature

((),
 {'input_text': TensorSpec(shape=(None,), dtype=tf.string, name='input_text')})

In [95]:
serving_default.structured_outputs

{'label_lookup': TensorSpec(shape=(None,), dtype=tf.string, name='label_lookup')}

In [96]:
serving_default(tf.constant(['talk to an agent']))

{'label_lookup': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'other-other'], dtype=object)>}

### original preprocessing logic 
Refer - https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/1    
This had an issue while loading the model for serving. It was still pointing to original vocab

In [424]:
tf.keras.backend.clear_session()

In [425]:
preprocessor_hub_url = bert_preprocessor_url
model_hub_url = bert_hub_url
max_top_intents = 2
checkpoint_path=tf.train.latest_checkpoint(output_folder)
bert_preprocessor_loaded = hub.load(preprocessor_hub_url)
input_text = tf.keras.layers.Input(shape=(), dtype=tf.string,name='input_text')
tokenize = hub.KerasLayer(bert_preprocessor_loaded.tokenize)
tokenized_inputs = [tokenize(input_text)] # bert can handle 2 input_texts (segments)

bert_pack_inputs = hub.KerasLayer(bert_preprocessor_loaded.bert_pack_inputs,arguments=dict(seq_length=max_seq_length))
encoder_inputs = bert_pack_inputs(tokenized_inputs)
encoder = hub.KerasLayer(model_hub_url,trainable=False)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]      # [batch_size, 768].
sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 768].
initializer = tf.keras.initializers.TruncatedNormal(stddev=bert_config["initializer_range"])
dense_output = tf.keras.layers.Dense(num_labels,kernel_initializer=initializer,name='output',dtype=tf.float32)(pooled_output)

# calculating softmax over the output layer
probs = tf.nn.softmax(dense_output,axis=-1,name='scores')
# sorting & taking only max_top_intents
sorted_indexes = tf.cast(tf.argsort(probs,axis=-1,direction='DESCENDING'),tf.int64)[:,:max_top_intents]
sorted_scores1 = tf.sort(probs,axis=-1,direction='DESCENDING',name="scores1")[:,:max_top_intents]
# this identity operation was only required to get the name correctly as in tf2.3 the sort object's name was coming differently
# https://github.com/tensorflow/tensorflow/issues/39398
# https://stackoverflow.com/questions/38626424/how-to-assign-new-name-or-rename-an-existing-tensor-in-tensorflow
sorted_scores = tf.identity(sorted_scores1,name='scores_sorted')

labels = LabelLookup(filepath=intent_file,default=default_label,name="classes")(sorted_indexes)

# loaded_model.compile()
#     checkpoint = tf.train.Checkpoint(model=model)
#     checkpoint.restore(checkpoint_path).assert_existing_objects_matched()

model  = tf.keras.Model(inputs={'input_text':input_text},
                     outputs={"classes":labels,"scores":sorted_scores})
# FIXME the following is a hack (not sure if it will create other issues)
# model.output_names = ['classes', 'scores']

#     checkpoint = tf.train.Checkpoint(model=model)
#     checkpoint.restore(checkpoint_path).assert_existing_objects_matched()
model.load_weights(checkpoint_path).assert_existing_objects_matched()

11 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7f4bd493df70> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/tutorials/customization/performance#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
11 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7f4bd48585e0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop,

In [426]:
tokenized_inputs

[tf.RaggedTensor(values=tf.RaggedTensor(values=Tensor("keras_layer/StatefulPartitionedCall:0", shape=(None,), dtype=int32), row_splits=Tensor("keras_layer/StatefulPartitionedCall:2", shape=(None,), dtype=int64)), row_splits=Tensor("keras_layer/StatefulPartitionedCall:1", shape=(None,), dtype=int64))]